In [60]:
import numpy as np
import math
from scipy.stats import poisson, norm, bernoulli, gamma
from scipy.special import psi

#nneuron = 41
#sprefs = np.linspace(-60, 60, nneuron)

nneuron = 5
sprefs = np.linspace(-20, 20, nneuron)

sig0_sq      = float(3**2)
sig1_sq      = float(12**2)
sigtc_sq     = float(10**2)
ndatapergain = 3000
mu = 0
gain = 15

def generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, gain, noise):
    s = np.random.normal(mu, np.sqrt(sig0_sq), ndatapergain)
    sprefs_data = np.tile(sprefs, (ndatapergain, 1))
    r = gain * np.exp(-np.square((np.transpose(np.tile(s, (nneuron, 1))) - sprefs_data))/(2 * sigtc_sq))
    if noise == "poisson":
        r = np.random.poisson(r) + 0.0
    return r, s

In [142]:
#simple version
r, actual = generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, 1, "poisson")
def norm_g(zTw):
    return zTw + 3

def relu(zTw):
    return max(0.000000000000001, zTw)

def g_norm(z, loc, scale):
    d_loc = (z - loc)/scale
    d_scale = -(1/(2 * scale)) + ((z - loc)**2)/(2 * scale**2)
    return [d_loc, d_scale]

L = 1
s = 1000
z = [None]*L
W = [None]*L
p_z = [None]*(L + 1)
q_z = [None]*L
p_W = [None]*L
q_W = [None]*L
g = [None]*L
g_W = [None]*L

xi_normal = np.random.rand(nneuron, 2) #[mu, sigma]
#train_data = r[1:3000]
train_data = [r[1]]
actual = actual[1:3000]
rho = .001
rhoW = .01

num = 1
for i in range(len(train_data)):
    n = train_data[i]
    true = actual[i]
    converged = False
    lam_normal = np.random.rand(1, 2)[0] * 3 #[mu, sigma]
    while not converged:
        #normal
        z_loc, z_scale = lam_normal
        z[0] = np.random.normal(loc=z_loc, scale=z_scale, size=s)
        W[0] = [None]*len(xi_normal)
        for i in range(len(xi_normal)):
            W_loc, W_scale = xi_normal[i]
            W[0][i] = np.random.normal(loc=W_loc, scale=W_scale, size=s).tolist()
        p_z[0] = [None] * s
        p_z[1] = [None] * s
        q_z[0] = [None] * s
        g[0] = [None] * s
        g_W[0] = [None] * s
        params = z[0] * W[0]
        params_a = np.array(params)
        for t in range(s):
            lam_p = params_a[:, t]
            p_z[0][t] = [None] * len(lam_p)
            g_W[0][t] = [None] * len(lam_p)
            for i in range(len(n)):
                p_z[0][t][i] = poisson.pmf(n[i], lam_p[i])
                g_W[0][t][i] = g_norm(W[0][i], W_loc, W_scale)
            np.concatenate(g_W[0][t])
            q_z[0][t] = norm.pdf(x=z[0][t], loc=z_loc, scale=z_scale)
            g[0][t] = g_norm(z[0][t], z_loc, z_scale)
            p_z[1][t] = [norm.pdf(x=z[0][t], loc=mu, scale=np.sqrt(sig0_sq))]
        p_z[0] = np.log(p_z[0])
        p_z[1] = np.log(p_z[1])
        q_z[0] = np.log(q_z[0])
        
        l = 0
        S = [None] * L
        T = [None] * L
        for l in range(L):
            S[l] = [None] * len(z[l])
            T[l] = [None] * len(W[l])
            pzl = np.sum(p_z[l])
            for k in range(len(z[l])):
                glk = np.asarray(g[l][k])
                S[l][k] = glk * (pzl + p_z[l+1][k] + q_z[l][k]) #???
            for j in range(len(W[l])):
                gWlj = np.asarray(g_W[l][j])
                #T[l][j] = gWlj * pzl #???
                T[l][j] = gWlj * p_z[l][j] #???
                np.concatenate(T[l][j])
        lam_normal += list(rho * S[0][0])
        lam_normal[1] = relu(lam_normal[1])
        print "data", n
        print "current", xi_normal
        print "sampled params", params
        print "prob", p_z
        print "update", T
        for m in range(len(xi_normal)):
            rT0n = rhoW * T[0][m]
            rT0n = [rT0n[0][0], rT0n[1][0]]
            xi_normal[m] += rT0n
            xi_normal[m][1] = relu(xi_normal[m][1])
        num += 1
        if num > 1000:
            converged = True
            print "x", n
            print "lambda", lam_normal
            print "true", true
            print "xi", xi_normal
            num = 1
            rhoW = rhoW * .9
            rho = .001

first [[0.2089440928386396, nan, 0.03600724255297947, nan, nan], [0.30004101362251151, 0.70572886796472623, 0.061898707680486768, nan, 0.68367762642660146], [0.78811195538301115, 0.44063637126289734, 0.052187204356923665, 0.43161020106421655, 0.73870003590078381], [0.3159334163296334, 0.31492662801109433, 0.0097335154128067626, 0.40555858600061712, 0.73252342686555794], [0.79179071405611645, 0.55627358303721486, 0.045681468678992483, nan, nan], [0.49597195085026208, 0.5274049225565508, 0.085121783294330219, nan, nan], [0.5405826867269008, 0.42534586336794539, 0.0048739174610115494, 0.47116699245138854, 0.68189598134672014], [0.15191367454450361, 0.4563419074306122, nan, 0.83725073025370833, nan], [0.34548664581975108, 0.97848850027899947, nan, nan, nan], [0.25619727247754992, 0.1640441576891663, nan, 0.47229023075084658, 0.79004394093083385], [0.50836732248453442, nan, nan, 0.51927148439286464, 0.57677562783291336], [0.30449786587333399, 0.977872311335235, 0.1226477386850649, 0.6410159

ValueError: operands could not be broadcast together with shapes (5,2,1000) (5,) 